# Test Modules

In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="__array__ implementation doesn't accept a copy keyword"
)
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'
 
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_testing_notebook, enable_autoreload, run_silent_subprocess

# Enable mixed precision for GPU training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Enable autoreload for development
enable_autoreload()

# Set up environment
env = setup_testing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
print_resource_summary = env['print_resource_summary']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<pre>Autoreload enabled</pre>

<pre>Project root: /scratch/edk202/word2gm-fast
TensorFlow version: 2.19.0
Device mode: GPU-enabled</pre>

<pre>Testing environment ready!</pre>

In [5]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
============================================================
Hostname: gr045.hpc.nyu.edu

Job Allocation:
   CPUs: 14
   Memory: 125.0 GB
   Requested partitions: rtx8000,v100,a100_2,a100_1,h100_1
   Actually running on: SSH failed: Host key verification failed.
   Job ID: 62906937
   Node list: gr045

GPU Information:
   CUDA GPUs detected: 1
   GPU 0: Quadro RTX 8000
      Memory: 0.5/45.0 GB (44.5 GB free)
      Temperature: 30°C
      Utilization: GPU 0%, Memory 0%

TensorFlow GPU Detection:
   TensorFlow detects 1 GPU(s)
      /physical_device:GPU:0, Memory growth: True
   Built with CUDA: True
============================================================</pre>

In [6]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="__array__ implementation doesn't accept a copy keyword"
)

import subprocess

result = subprocess.run([
    'pytest',
    'tests/'
], capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collected 37 items

tests/test_corpus_to_dataset.py .                                        [  2%]
tests/test_dataset_to_triplets.py ......                                 [ 18%]
tests/test_index_vocab.py ..                                             [ 24%]
tests/test_notebook_training.py .                                        [ 27%]
tests/test_resource_monitor.py ..                                        [ 32%]
tests/test_tfrecord_io.py .......                                        [ 51%]
tests/test_train_loop.py .                                               [ 54%]
tests/test_training_utils.py ....                                        [ 64%]
tests/test_word2gm_model.py .............                                [100%]

=============================== warnings su